In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
opt_df = df.drop(['EIN', 'NAME'], axis=1)
opt_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
opt_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = opt_df['APPLICATION_TYPE'].value_counts().loc[opt_df['APPLICATION_TYPE'].value_counts() < 100].index

# Replace in dataframe
for app in application_types_to_replace:

    opt_df['APPLICATION_TYPE'] = opt_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
opt_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
opt_df['CLASSIFICATION'].value_counts() > 1

C1000     True
C2000     True
C1200     True
C3000     True
C2100     True
         ...  
C4120    False
C8210    False
C2561    False
C4500    False
C2150    False
Name: CLASSIFICATION, Length: 71, dtype: bool

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = opt_df['CLASSIFICATION'].value_counts().loc[opt_df['CLASSIFICATION'].value_counts() < 400].index

# Replace in dataframe
for cls in classifications_to_replace:
    opt_df['CLASSIFICATION'] = opt_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
opt_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
model_df = pd.get_dummies(opt_df)
model_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
X = model_df.drop("IS_SUCCESSFUL", axis=1)
y = model_df.IS_SUCCESSFUL

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

print(X_train.shape)
print(X_test.shape)

(25724, 45)
(8575, 45)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=len(X.columns)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 230       
                                                                 
 dense_1 (Dense)             (None, 3)                 18        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 252 (1008.00 Byte)
Trainable params: 252 (1008.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 11s 7ms/step - loss: 0.6499 - accuracy: 0.6597
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5941 - accuracy: 0.7148
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5797 - accuracy: 0.7253
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5746 - accuracy: 0.7262
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5721 - accuracy: 0.7264
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5705 - accuracy: 0.7282
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5690 - accuracy: 0.7284
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5681 - accuracy: 0.7285
Epoch 9/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5676 - accuracy: 0.7275
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5671 - accuracy: 0.728

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5676 - accuracy: 0.7276 - 591ms/epoch - 2ms/step
Loss: 0.5676316022872925, Accuracy: 0.727580189704895


In [ ]:
# Save best model as h5 file
nn.save('AlphabetSoupCharity.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn2= tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=len(X.columns)))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=3, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 5)                 230       
                                                                 
 dense_4 (Dense)             (None, 3)                 18        
                                                                 
 dense_5 (Dense)             (None, 2)                 8         
                                                                 
 dense_6 (Dense)             (None, 1)                 3         
                                                                 
Total params: 259 (1.01 KB)
Trainable params: 259 (1.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=75)

Epoch 1/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5633 - accuracy: 0.7303
Epoch 2/75
804/804 [==============================] - 3s 3ms/step - loss: 0.5629 - accuracy: 0.7317
Epoch 3/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5630 - accuracy: 0.7311
Epoch 4/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5629 - accuracy: 0.7316
Epoch 5/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5628 - accuracy: 0.7311
Epoch 6/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5629 - accuracy: 0.7318
Epoch 7/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5628 - accuracy: 0.7315
Epoch 8/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5628 - accuracy: 0.7309
Epoch 9/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5628 - accuracy: 0.7316
Epoch 10/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5628 - accuracy: 0.7316

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7891 - accuracy: 0.4208 - 587ms/epoch - 2ms/step
Loss: 0.7891204953193665, Accuracy: 0.4207580089569092


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn3= tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=len(X.columns)))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 5)                 230       
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 236 (944.00 Byte)
Trainable params: 236 (944.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn3.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6402 - accuracy: 0.6642
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5687 - accuracy: 0.7241
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5621 - accuracy: 0.7246
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5591 - accuracy: 0.7255
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5578 - accuracy: 0.7254
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5573 - accuracy: 0.7260
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7261
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7263
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5559 - accuracy: 0.7262
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7253

In [ ]:
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5553 - accuracy: 0.7296 - 753ms/epoch - 3ms/step
Loss: 0.5552874803543091, Accuracy: 0.7295626997947693


In [ ]:
opt_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
opt2_df = opt_df.drop(['SPECIAL_CONSIDERATIONS'], axis=1)
opt2_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = opt2_df['APPLICATION_TYPE'].value_counts().loc[opt2_df['APPLICATION_TYPE'].value_counts() < 100].index

# Replace in dataframe
for app in application_types_to_replace:

    opt2_df['APPLICATION_TYPE'] = opt2_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
opt2_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = opt2_df['CLASSIFICATION'].value_counts().loc[opt2_df['CLASSIFICATION'].value_counts() < 400].index

# Replace in dataframe
for cls in classifications_to_replace:
    opt2_df['CLASSIFICATION'] = opt2_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
opt2_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
opt2_df = pd.get_dummies(opt2_df)
opt2_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [ ]:
# Split our preprocessed data into our features and target arrays
X = opt2_df.drop("IS_SUCCESSFUL", axis=1)
y = opt2_df.IS_SUCCESSFUL

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

print(X_train.shape)
print(X_test.shape)

(25724, 43)
(8575, 43)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=len(X.columns)))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=3, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 5)                 220       
                                                                 
 dense_10 (Dense)            (None, 3)                 18        
                                                                 
 dense_11 (Dense)            (None, 1)                 4         
                                                                 
Total params: 242 (968.00 Byte)
Trainable params: 242 (968.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn4.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.6325 - accuracy: 0.6501
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5698 - accuracy: 0.7237
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5606 - accuracy: 0.7250
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5582 - accuracy: 0.7252
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5568 - accuracy: 0.7262
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5560 - accuracy: 0.7269
Epoch 7/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5551 - accuracy: 0.7270
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7275
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5541 - accuracy: 0.7273
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7280

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5544 - accuracy: 0.7270 - 465ms/epoch - 2ms/step
Loss: 0.5544444918632507, Accuracy: 0.7269970774650574


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn5 = tf.keras.models.Sequential()

# First hidden layer
nn5.add(tf.keras.layers.Dense(units=4, activation="relu", input_dim=len(X.columns)))

# Output layer
nn5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 4)                 176       
                                                                 
 dense_13 (Dense)            (None, 1)                 5         
                                                                 
Total params: 181 (724.00 Byte)
Trainable params: 181 (724.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn5.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6253 - accuracy: 0.6811
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5803 - accuracy: 0.7238
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5740 - accuracy: 0.7243
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5705 - accuracy: 0.7248
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5681 - accuracy: 0.7248
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5667 - accuracy: 0.7250
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5658 - accuracy: 0.7252
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5652 - accuracy: 0.7257
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5622 - accuracy: 0.7252
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5594 - accura

In [ ]:
model_loss, model_accuracy = nn5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5567 - accuracy: 0.7251 - 777ms/epoch - 3ms/step
Loss: 0.5567286014556885, Accuracy: 0.7251312136650085


In [ ]:
# Unfortunately, after 5 different optimizations we were not able to achieve 75% accuracy. WOMP WOMP :(
